<a href="https://colab.research.google.com/github/sammed02/Prepinsta-Winter-internship/blob/main/Winter_internship_week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/Week 5 - SQL Murder Mystery/sql-murder-mystery.db')
cursor = conn.cursor()

In [ ]:
print(conn)


In [ ]:
print(cursor.fetchall())

[]


In [ ]:
task_1 = "SELECT * FROM 'crime_scene_report' WHERE date = '20180115' and city='SQL City' ;"

In [ ]:
pd.set_option('display.max_colwidth', None)
task1 = pd.read_sql_query(task_1, conn)
task1

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [ ]:
task1.shape[0]

3

In [ ]:
#task 1
cursor.execute("SELECT * FROM 'crime_scene_report' WHERE date = '20180115' and city='SQL City' ;")
crime_scene_report = cursor.fetchall()
print("Info about crime scene :", crime_scene_report)
print(len(crime_scene_report))

Info about crime scene : [(20180115, 'assault', 'Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!', 'SQL City'), (20180115, 'assault', 'Report Not Found', 'SQL City'), (20180115, 'murder', 'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".', 'SQL City')]
3


In [ ]:
# task2
# Witnesses : Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr".
# The second witness, named Annabel, lives somewhere on "Franklin Ave".
witness1 = '''SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number desc;'''
witness2 = '''
  SELECT * FROM person WHERE name like "Annabel%"
  AND address_street_name = "Franklin Ave";

'''
task2 = pd.read_sql_query(witness1, conn)[:1]
task2

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [ ]:
task2 = pd.read_sql_query(witness2, conn)
task2

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [ ]:
#person_id is id of Morty and Annabel to get their interview
task_3 = '''
  SELECT * FROM interview WHERE person_id = 14887 OR person_id = 16371;
'''

In [ ]:
task3 = pd.read_sql_query(task_3, conn)
task3


,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [ ]:
# gym membership id is mentioned in interview of morty and date is mentioned in annabel interview
task_4 = '''SELECT *
FROM get_fit_now_check_in
WHERE membership_id like "%48Z%" AND check_in_date = 20180109
order by check_in_date;'''

In [ ]:
task4 = pd.read_sql_query(task_4, conn)
task4

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [ ]:
# we found 2 suspects , lets check car details which we noted from interview
task_5 = '''SELECT *
FROM drivers_license
WHERE plate_number like "%H42W%";'''

In [ ]:
task5 = pd.read_sql_query(task_5,conn)
task5

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [ ]:
#checking personal details of both the males from the above task
task_6 = '''
SELECT *
FROM person
WHERE license_id = "423327" OR license_id = "664760";
'''
task6 = pd.read_sql_query(task_6, conn)
task6

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [ ]:
task_7 = '''
SELECT *
FROM get_fit_now_member
WHERE person_id = "51739" OR person_id = "67318";
'''
task7 = pd.read_sql_query(task_7, conn)
task7

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


In [ ]:
# task8 :::: As per witness murderer hold gold membershio status
# So we found JEREMY BOWERS as a murderer


In [ ]:
# get info of murder
task_9 = '''SELECT *
FROM interview
WHERE person_id = 67318;'''

In [ ]:
task9 = pd.read_sql_query(task_9, conn)
task9

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [ ]:
# get info about woman mentioned
task_10 = '''SELECT *
FROM drivers_license
WHERE car_make = "Tesla" AND car_model = "Model S" AND
gender = "female" AND hair_color = "red";'''


In [ ]:
task10 = pd.read_sql_query(task_10, conn)
task10

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [ ]:
task_11 = '''SELECT *
FROM person
WHERE license_id = "202298" OR license_id = "291182" OR license_id = "918773";'''

In [ ]:
task11 = pd.read_sql_query(task_11,conn)
task11

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,90700,Regina George,291182,332,Maple Ave,337169072
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388
